# le get et post ( Recup de secret dans coffre fort "keyvolts")

In [10]:
# 1. Import des bibliothèques nécessaires
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import os
import io

from dotenv import load_dotenv




In [11]:

# 2. Chargement des variables d'environnement
load_dotenv()

# 2. Configuration des informations d'authentification
# Configuration Key Vault
key_vault_url = os.getenv('KEY_VAULT_URL')
secret_name = os.getenv('WRITE_SECRET_NAME')

# Configuration Azure AD
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('WRITE_CLIENT_ID')

# Configuration Storage
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
container_name = os.getenv('CONTAINER_NAME')


In [12]:

# 3. Récupération du secret depuis Key Vault
def get_secret_from_keyvault():
    try:
        # Utilisation de DefaultAzureCredential pour l'authentification
        credential = DefaultAzureCredential()
        
        # Création du client Key Vault
        secret_client = SecretClient(vault_url=key_vault_url, credential=credential)
        
        # Récupération du secret
        secret = secret_client.get_secret(secret_name)
        return secret.value
        
    except Exception as e:
        print(f"Erreur lors de la récupération du secret: {str(e)}")
        return None

In [13]:
def create_credentials():
    client_secret = get_secret_from_keyvault()
    if client_secret:
        return ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )
    return None

In [14]:

# 5. Fonction pour uploader le fichier vers Data Lake
def upload_to_datalake(local_file_path, destination_path):
    try:
        # Obtention des credentials
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        # URL du service
        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        
        # Création du client service
        service_client = DataLakeServiceClient(account_url, credential=credential)
        
        # Obtention du client système de fichiers
        file_system_client = service_client.get_file_system_client(file_system=container_name)
        
        # Création du client fichier
        file_client = file_system_client.create_file(destination_path)
        
        # Lecture et upload du fichier
        with open(local_file_path, 'rb') as local_file:
            file_contents = local_file.read()
            
        file_client.append_data(data=file_contents, offset=0, length=len(file_contents))
        file_client.flush_data(len(file_contents))
            
        print(f"Fichier uploadé avec succès vers: {destination_path}")
        
    except Exception as e:
        print(f"Une erreur est survenue: {str(e)}")

In [15]:
# 6. Exécution du script
if __name__ == "__main__":
    # Chemin du fichier local
    local_file_path = "DATA/reviews.csv"
    
    # Chemin de destination dans Data Lake
    destination_file_path = "reviews.csv"
    
    # Upload du fichier
    upload_to_datalake(local_file_path, destination_file_path)

Fichier uploadé avec succès vers: reviews.csv


# Lecture du CSV

In [9]:
# # 7. Fonction pour lire les premières lignes d'un fichier CSV depuis le Data Lake
# def read_first_lines_from_datalake(file_name, n_lines=10):
#     try:
#         # Obtention des credentials
#         credential = create_credentials()
#         if not credential:
#             raise Exception("Impossible d'obtenir les credentials")

#         # URL du service
#         account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        
#         # Création du client service
#         service_client = DataLakeServiceClient(account_url, credential=credential)
        
#         # Obtention du client système de fichiers
#         file_system_client = service_client.get_file_system_client(container_name)
        
#         # Obtention du client fichier
#         file_client = file_system_client.get_file_client(file_name)
        
#         # Téléchargement du contenu
#         download = file_client.download_file()
#         content = download.readall()
        
#         # Conversion en DataFrame pandas
#         df = pd.read_csv(io.BytesIO(content))
        
#         print(f"\n📄 Premières {n_lines} lignes du fichier {file_name}:")
#         print("=" * 80)
#         print(df.head(n_lines))
        
#         print("\n📊 Informations sur le fichier:")
#         print("=" * 80)
#         print(f"Nombre total de lignes: {len(df)}")
#         print(f"Nombre de colonnes: {len(df.columns)}")
#         print("\nColonnes disponibles:")
#         for col in df.columns:
#             print(f"- {col}")
            
#         return df.head(n_lines)
        
#     except Exception as e:
#         print(f"❌ Une erreur est survenue: {str(e)}")
#         return None

# # 8. Exécution de la lecture
# if __name__ == "__main__":
#     import io  # Ajoutez cet import en haut du fichier si pas déjà présent
    
#     # Lecture des 10 premières lignes du fichier
#     file_name = "reviews.csv"
#     first_lines = read_first_lines_from_datalake(file_name)
    
#     if first_lines is not None:
#         # Optionnel : Sauvegarde des 10 premières lignes en local
#         first_lines.to_csv("preview_reviews.csv", index=False)
#         print("\n💾 Les 10 premières lignes ont été sauvegardées dans 'preview_reviews.csv'")



📄 Premières 10 lignes du fichier reviews.csv:
   listing_id        date
0      252232  2012-03-02
1      252232  2012-05-22
2      252232  2012-06-11
3      252232  2012-07-28
4      252232  2012-09-02
5      252232  2013-05-11
6      252232  2013-05-21
7      252232  2013-05-28
8      252232  2013-06-18
9      252232  2013-08-21

📊 Informations sur le fichier:
Nombre total de lignes: 927474
Nombre de colonnes: 2

Colonnes disponibles:
- listing_id
- date

💾 Les 10 premières lignes ont été sauvegardées dans 'preview_reviews.csv'


# Ajouter une ligne 

In [ ]:
# 9. Fonction pour ajouter une ligne au fichier CSV
def append_row_to_csv(file_name, new_listing_id, new_date):
    try:
        # Obtention des credentials
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        # URL du service
        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        
        # Création du client service
        service_client = DataLakeServiceClient(account_url, credential=credential)
        
        # Obtention du client système de fichiers
        file_system_client = service_client.get_file_system_client(container_name)
        
        # Obtention du client fichier
        file_client = file_system_client.get_file_client(file_name)
        
        # Lecture du fichier existant
        download = file_client.download_file()
        content = download.readall()
        
        # Conversion en DataFrame
        df = pd.read_csv(io.BytesIO(content))
        
        # Création de la nouvelle ligne
        new_row = pd.DataFrame({
            'listing_id': [new_listing_id],
            'date': [new_date]
        })
        
        # Ajout de la nouvelle ligne
        df_updated = pd.concat([df, new_row], ignore_index=True)
        
        # Conversion du DataFrame en bytes
        csv_content = df_updated.to_csv(index=False).encode('utf-8')
        
        # Création d'un nouveau fichier (écrase l'ancien)
        file_client = file_system_client.create_file(file_name)
        file_client.append_data(data=csv_content, offset=0, length=len(csv_content))
        file_client.flush_data(len(csv_content))
        
        print(f"✅ Nouvelle ligne ajoutée avec succès!")
        print("\n📊 Aperçu des dernières lignes après ajout:")
        print(df_updated.tail())
        
        return df_updated
        
    except Exception as e:
        print(f"❌ Une erreur est survenue: {str(e)}")
        return None

# 10. Exécution de l'ajout de ligne
if __name__ == "__main__":
    # Fichier à modifier
    file_name = "reviews.csv"
    
    # Nouvelles valeurs à ajouter
    new_listing_id = 666
    new_date = "1988-04-02"
    
    # Ajout de la nouvelle ligne
    print(f"\nAjout d'une nouvelle ligne avec listing_id={new_listing_id} et date={new_date}")
    updated_df = append_row_to_csv(file_name, new_listing_id, new_date)
    
    if updated_df is not None:
        # Vérification du nombre total de lignes
        print(f"\nNombre total de lignes après ajout: {len(updated_df)}")



Ajout d'une nouvelle ligne avec listing_id=666 et date=1988-04-02
✅ Nouvelle ligne ajoutée avec succès!

📊 Aperçu des dernières lignes après ajout:
                 listing_id        date
927470  1226644265166272585  2024-08-25
927471  1226644265166272585  2024-09-03
927472  1227947729562300326  2024-08-22
927473  1227947729562300326  2024-09-05
927474                  666  1988-04-02

Nombre total de lignes après ajout: 927475


# ⚠️  Test de suppression de fichier  ⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️

In [ ]:
# # 7. Fonction pour supprimer un fichier du Data Lake
# def delete_from_datalake(file_path):
#     try:
#         # Obtention des credentials
#         credential = create_credentials()
#         if not credential:
#             raise Exception("Impossible d'obtenir les credentials")

#         # URL du service
#         account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        
#         # Création du client service
#         service_client = DataLakeServiceClient(account_url, credential=credential)
        
#         # Obtention du client système de fichiers
#         file_system_client = service_client.get_file_system_client(file_system=container_name)
        
#         # Obtention du client fichier
#         file_client = file_system_client.get_file_client(file_path)
        
#         # Tentative de suppression
#         print(f"\nTentative de suppression du fichier {file_path}...")
#         file_client.delete_file()
#         print(f"✅ Le fichier {file_path} a été supprimé avec succès!")
        
#     except Exception as e:
#         print("\n❌ Résultat du test de suppression:")
#         print("-" * 50)
#         print(f"Une erreur est survenue: {str(e)}")
        
#         if "AuthorizationPermissionMismatch" in str(e):
#             print("\n⚠️ Vous n'avez pas les droits de suppression!")
#         else:
#             print("\nUne erreur inattendue s'est produite.")

In [ ]:


# # 8. Fonction pour vérifier si le fichier existe
# def check_file_exists(file_path):
#     try:
#         credential = create_credentials()
#         if not credential:
#             raise Exception("Impossible d'obtenir les credentials")

#         account_url = f"https://{storage_account_name}.dfs.core.windows.net"
#         service_client = DataLakeServiceClient(account_url, credential=credential)
#         file_system_client = service_client.get_file_system_client(file_system=container_name)
#         file_client = file_system_client.get_file_client(file_path)
        
#         # Tente d'obtenir les propriétés du fichier
#         file_client.get_file_properties()
#         return True
#     except:
#         return False


In [ ]:

# # 9. Test de suppression et vérification
# if __name__ == "__main__":
#     # Fichier à supprimer
#     file_to_delete = "reviews.csv"
    
#     # Vérifie d'abord si le fichier existe
#     print(f"\nVérification de l'existence du fichier {file_to_delete}...")
#     if check_file_exists(file_to_delete):
#         print(f"📄 Le fichier {file_to_delete} existe.")
#         # Tente la suppression
#         delete_from_datalake(file_to_delete)
        
#         # Vérifie après la suppression
#         if not check_file_exists(file_to_delete):
#             print(f"✅ Vérification finale: Le fichier {file_to_delete} n'existe plus.")
#         else:
#             print(f"⚠️ Vérification finale: Le fichier {file_to_delete} existe toujours.")
#     else:
#         print(f"❌ Le fichier {file_to_delete} n'existe pas dans le conteneur.")
 